In [55]:
import pandas as pd
from os import getcwd

In [56]:
path = getcwd()
df = pd.read_csv(path+'/../output/results_df.csv')

In [57]:
df.shape
df.head()

,game_date,team,opponent,record,1st_inning,2nd_inning,3rd_inning,4th_inning,5th_inning,6th_inning,7th_inning,8th_inning,9th_inning,runs,hits,errors,result
0,2023-02-25,Mets,Astros,test,2,0,0,0,0,0,0,0,0,2,4,4,test
1,2023-02-25,Astros,Mets,test,0,0,0,0,0,1,3,0,X,4,8,0,test
2,2023-02-25,Nationals,Cardinals,test,1,0,0,0,0,0,0,1,1,3,9,1,test
3,2023-02-25,Cardinals,Nationals,test,0,0,2,0,0,0,0,0,0,2,5,0,test
4,2023-02-25,Blue Jays,Pirates,test,0,0,1,3,2,0,3,0,0,9,13,1,test


In [72]:
df[['game_date']].drop_duplicates()

,game_date
0,2023-02-25
34,2023-02-26
66,2023-02-28
98,2023-03-01
128,2023-03-02
158,2023-03-03
190,2023-03-04
226,2023-03-05


In [73]:
df.groupby(by=df['team'])['runs'].sum().sort_values(ascending=False)

team
Royals       75
Reds         68
Athletics    67
Blue Jays    67
Yankees      56
Twins        56
Red Sox      56
Rangers      56
Padres       55
Mets         55
D-backs      54
Rockies      52
Tigers       50
Brewers      49
Guardians    49
Rays         49
Angels       46
Giants       46
Braves       46
Astros       45
Phillies     44
Dodgers      40
Cubs         40
Mariners     38
Nationals    38
Orioles      36
Pirates      33
Cardinals    33
White Sox    33
Marlins      23
Name: runs, dtype: int64

In [75]:
df.groupby(by=df['team'])['hits'].sum().sort_values(ascending=False)

team
Royals       110
Athletics    100
Reds          96
Blue Jays     95
Rangers       88
D-backs       87
Giants        85
Yankees       81
Rockies       81
Brewers       80
Guardians     79
Mariners      79
Braves        78
Mets          78
Tigers        76
Angels        75
Nationals     75
Padres        74
Phillies      74
Twins         74
Astros        73
Red Sox       71
Cardinals     70
White Sox     70
Rays          69
Cubs          68
Dodgers       64
Orioles       62
Marlins       57
Pirates       50
Name: hits, dtype: int64

In [76]:
df.groupby(by=df['team'])['errors'].sum().sort_values(ascending=False)

team
Twins        13
D-backs      12
Mets         12
Giants       12
White Sox    11
Angels        9
Blue Jays     9
Rays          9
Phillies      9
Mariners      8
Pirates       8
Marlins       8
Nationals     8
Brewers       8
Braves        8
Rangers       7
Athletics     7
Tigers        7
Red Sox       7
Reds          6
Guardians     6
Cardinals     6
Yankees       6
Dodgers       5
Rockies       5
Royals        4
Astros        4
Cubs          3
Padres        3
Orioles       3
Name: errors, dtype: int64

In [74]:
df[df['team']=='Royals']

,game_date,team,opponent,record,1st_inning,2nd_inning,3rd_inning,4th_inning,5th_inning,6th_inning,7th_inning,8th_inning,9th_inning,runs,hits,errors,result
22,2023-02-25,Royals,Rangers,test,0,0,0,7,1,0,1,0,1,10,10,0,test
53,2023-02-26,Royals,Mariners,test,5,0,0,0,0,3,0,0,X,8,11,0,test
80,2023-02-28,Royals,Guardians,test,0,0,3,0,0,2,0,1,6,12,15,0,test
124,2023-03-01,Royals,Rockies,test,0,0,0,2,0,0,1,0,5,8,11,2,test
145,2023-03-02,Royals,Angels,test,2,0,0,4,0,0,0,2,X,8,15,0,test
170,2023-03-03,Royals,Athletics,test,0,2,2,1,0,0,1,0,0,6,9,1,test
208,2023-03-04,Royals,Reds,test,2,1,1,0,1,0,3,2,2,12,14,0,test
225,2023-03-04,Royals,Dodgers,test,1,0,0,0,0,2,0,2,1,6,14,1,test
244,2023-03-05,Royals,Giants,test,1,0,0,3,1,0,0,0,0,5,11,0,test
